# Simulation remplissage des cuves de Besançon

In [1]:
import calendar
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label, Layout
import plotly.graph_objects as go

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import plotly.io as pio
pio.renderers.default = "notebook"

In [2]:
calendar_month = {id_month+1: month for id_month, month in enumerate([m for m in list(calendar.month_name) if len(m) > 0])}
reversed_calendar_month = {v: k for k, v in calendar_month.items()}

In [3]:
def estimation_eau_collectee(df):
    df['eau collectées ce jour (cuves)'] = df['pluviometrie'].apply(lambda pluviometrie: (1-(w_percent_perte_cuves.value / 100.0))*((pluviometrie*w_m2_garage.value)/1000.0))
    df.loc[df['mois'].isin(get_months_between_in_widget(w_mois_vides_cuves)), ['eau collectées ce jour (cuves)']] = 0
    df['eau collectées ce jour (citerne)'] = df['pluviometrie'].apply(lambda pluviometrie: (pluviometrie*w_m2_maison.value)/1000.0)

def estimation_conso_jardin(df):
    df["quantité d'eau vitale"] = df["pluviometrie"].apply(lambda eau: 1 if eau > w_min_pluviometrie_vitale.value else 0)
    arrosages = []
    nombre_jour_sans_eau = 1e-10
    for eau in df["quantité d'eau vitale"]:
        if eau == 0:
            nombre_jour_sans_eau += 1
        else:
            nombre_jour_sans_eau = 1e-10
        arrosages.append(min(np.around(arrosage(nombre_jour_sans_eau, w_nombre_maximum_periode_sans_eau.value), 4), 1))
    df["conso jardin (%)"] = arrosages
    #df.loc[df['mois'].isin(get_months_between_in_widget(w_mois_vides_cuves)), ['conso jardin (%)']] = 0

def simulation():
    global df
    # Eau collectée par jour en fonction de la pluviométrie et de la surface de toit
    estimation_eau_collectee(df)
    
    # Estimation du besoin en eau en fonction de la pluviométrie
    estimation_conso_jardin(df)
    # Calcul du reste dans la cuve après conso jardin
    eau_consommees = []
    eau_consommees_citerne_jardin = []
    eau_consommees_cuves_jardin = []
    eau_consommees_maison = []
    eau_restantes_cuves = []
    eau_restantes_citerne = []
    eau_consommees_secteur_jardin = []
    eau_consommees_secteur_maison = []
    eau_consommees_secteur = []
    pertes_trop_plein_cuves = []
    pertes_trop_plein_citerne = []

    reste_jour_precedent_cuves = 0
    reste_jour_precedent_citerne = w_reste_jour_precedent_citerne.value

    for _, row in df.iterrows():
        # cuves
        eau_collectee = row['eau collectées ce jour (cuves)']
        eau_dispo = (eau_collectee * min(m3_stockage_cuves, 1)) + reste_jour_precedent_cuves
        # jardin
        eau_consomme_cuves = min(row['conso jardin (%)'] * min(eau_dispo, besoin_eau_jardin_jour), besoin_eau(row['mois'], get_months_between_in_widget(w_mois_le_plus_consommateur), w_std_besoin_eau.value))
        reste = eau_dispo - eau_consomme_cuves
        perte_trop_plein = max(reste - m3_stockage_cuves, 0)
        if m3_stockage_cuves < eau_collectee:
            perte_trop_plein += (eau_collectee - m3_stockage_cuves)
        # reste
        reste = min(reste, m3_stockage_cuves)
        reste_jour_precedent_cuves = reste
        # append
        eau_restantes_cuves.append(reste)
        pertes_trop_plein_cuves.append(perte_trop_plein)
        eau_consommees_cuves_jardin.append(eau_consomme_cuves)

        # citerne
        eau_collectee = row['eau collectées ce jour (citerne)']
        eau_dispo = (eau_collectee * min(m3_stockage_citerne, 1)) + reste_jour_precedent_citerne
        # jardin
        eau_consomme_citerne_jardin = 0
        if w_citerne_pour_conso_jardin.value and reste == 0:
            part_arrosage_cuves = min((eau_consomme_cuves / besoin_eau_jardin_jour), 1)
            eau_consomme_citerne_jardin = min((1 - part_arrosage_cuves) * row['conso jardin (%)'] * min(eau_dispo, besoin_eau_jardin_jour), besoin_eau(row['mois'], get_months_between_in_widget(w_mois_le_plus_consommateur), w_std_besoin_eau.value))
        # maison
        eau_consomme_citerne_maison = 0
        if w_citerne_pour_conso_maison.value and eau_dispo > m3_moyen_journalier_maison:
            if row['mois'] not in get_months_between_in_widget(w_mois_citerne_priorite_jardin) or reste_jour_precedent_citerne > w_stock_faible_citerne.value:
                eau_consomme_citerne_maison = m3_moyen_journalier_maison
        # reste
        reste = eau_dispo - eau_consomme_citerne_jardin - eau_consomme_citerne_maison
        perte_trop_plein = max(reste - m3_stockage_citerne, 0)
        if m3_stockage_citerne < eau_collectee:
            perte_trop_plein += (eau_collectee - m3_stockage_citerne)
        reste = min(reste, m3_stockage_citerne)
        reste_jour_precedent_citerne = reste
        # append
        eau_restantes_citerne.append(reste)
        pertes_trop_plein_citerne.append(perte_trop_plein)
        eau_consommees_maison.append(eau_consomme_citerne_maison)
        eau_consommees_citerne_jardin.append(eau_consomme_citerne_jardin)

        # secteur
        eau_consomme_jardin = eau_consomme_cuves + eau_consomme_citerne_jardin
        # jardin
        eau_consommee_secteur_jardin = 0
        if row['pluviometrie'] < w_min_pluviometrie_vitale.value and eau_consomme_jardin < besoin_eau_jardin_jour: # and eau_dispo < w_stock_faible_cuves.value: # TODO select only eau dispo usable for jardin
            eau_consommee_secteur_jardin = besoin_eau(row['mois'], get_months_between_in_widget(w_mois_le_plus_consommateur), w_std_besoin_eau.value) * row["conso jardin (%)"] * (besoin_eau_jardin_jour - eau_consomme_cuves - eau_consomme_citerne_jardin)
        eau_consommees_secteur_jardin.append(eau_consommee_secteur_jardin)
        # maison
        eau_consommee_secteur_maison = max(m3_moyen_journalier_maison - eau_consomme_citerne_maison, 0)
        eau_consommees_secteur_maison.append(eau_consommee_secteur_maison)

        # global
        eau_consommees.append(eau_consomme_jardin + eau_consomme_citerne_maison)
        eau_consommees_secteur.append(eau_consommee_secteur_jardin + eau_consommee_secteur_maison)

    df['eau consommée pour le jardin (cuves)'] = eau_consommees_cuves_jardin
    df['eau consommée pour le jardin (citerne)'] = eau_consommees_citerne_jardin
    df['eau consommée pour le jardin'] = eau_consommees
    df['eau consommée pour la maison'] = eau_consommees_maison
    df['eau restante (cuves)'] = eau_restantes_cuves
    df['eau restante (citerne)'] = eau_restantes_citerne
    df['perte de trop plein (cuves)'] = pertes_trop_plein_cuves
    df['perte de trop plein (citerne)'] = pertes_trop_plein_citerne

    df['eau restante sans trop plein (cuves)'] = df.apply(lambda row: row['eau restante (cuves)'] + row['perte de trop plein (cuves)'], axis=1)
    df['eau restante sans trop plein (citerne)'] = df.apply(lambda row: row['eau restante (citerne)'] + row['perte de trop plein (citerne)'], axis=1)

    df['eau consommée du secteur'] = eau_consommees_secteur
    df['eau consommée du secteur pour le jardin'] = eau_consommees_secteur_jardin
    df['eau consommée du secteur pour la maison'] = eau_consommees_secteur_maison

    df.loc[df['mois'].isin(get_months_between_in_widget(w_mois_vides_cuves)), ['eau restante (cuves)']] = 0
    df.loc[df['mois'].isin(get_months_between_in_widget(w_mois_vides_cuves)), ['eau restante sans trop plein (cuves)']] = 0
    #df.loc[df['mois'].isin(get_months_between_in_widget(w_mois_vides_cuves)), ['eau consommée du secteur']] = 0

In [4]:
def get_plots_simulation():
    global df
    plots = []
    # figure_data.extend([i for i in go.Figure(go.Scatter(x=df["mois_jour"], y=df["eau consommée pour la maison"], mode="markers", opacity=.4, name="eau consommée pour la maison (citerne)")).to_dict()['data']])
    plots.append([i for i in px.histogram(df, x="mois_jour", y="eau collectées ce jour (cuves)").to_dict()['data']]) # color_discrete_sequence=['lightsalmon'] '#EB89B5'
    plots.append([i for i in px.histogram(df, x="mois_jour", y="eau collectées ce jour (citerne)").to_dict()['data']])
    plots.append([i for i in go.Figure(go.Scatter(x=df["mois_jour"], y=df["eau restante sans trop plein (cuves)"], mode="markers+lines", opacity=.4, name="trop plein (cuves)")).to_dict()['data']])
    plots.append([i for i in go.Figure(go.Scatter(x=df["mois_jour"], y=df["eau restante sans trop plein (citerne)"], mode="markers+lines", opacity=.4, name="trop plein (citerne)")).to_dict()['data']])
    plots.append([i for i in go.Figure(go.Scatter(x=df["mois_jour"], y=df["eau restante (cuves)"], mode="markers+lines", opacity=.3, name="eau restante (cuves)")).to_dict()['data']])
    plots.append([i for i in go.Figure(go.Scatter(x=df["mois_jour"], y=df["eau restante (citerne)"], mode="markers+lines", opacity=.3, name="eau restante (citerne)")).to_dict()['data']])
    plots.append([i for i in px.histogram(df, x="mois_jour", y="eau consommée pour le jardin").to_dict()['data']])
    plots.append([i for i in px.histogram(df, x="mois_jour", y="eau consommée du secteur pour le jardin").to_dict()['data']])
    return plots

def plot_simulation():
    global df
    figure_data=[]
    plots = get_plots_simulation()
    for plot in plots:
        figure_data.extend(plot)
    fig=go.FigureWidget(figure_data)
    fig.add_shape(type='line', x0=0, y0=m3_stockage_cuves, x1=len(df), y1=m3_stockage_cuves, line=dict(color='rgba(100, 100, 100, 0.2)'), xref='x', yref='y')
    fig.add_shape(type='line', x0=0, y0=m3_stockage_citerne, x1=len(df), y1=m3_stockage_citerne, line=dict(color='rgba(100, 100, 100, 0.2)'), xref='x', yref='y')
    fig.update_layout(barmode='stack', height=800)

    fig.update_traces(overwrite=True, marker={"opacity": 0.7}) 
    return fig

def print_metrics():
    global df
    return \
    "%.2f m3 d'eau collectée dans la citerne et les cuves" % (df['eau collectées ce jour (citerne)'].sum() + df['eau collectées ce jour (cuves)'].sum()) \
    + "</br>%.2f m3 du secteur pour le jardin et la maison" % df['eau consommée du secteur'].sum() \
    + '</br>===== Citerne' \
    + "</br>%.2f m3 d'eau collectée" % df['eau collectées ce jour (citerne)'].sum() \
    + "</br>%.2f m3 d'eau utilisés (%.2f m3 pour le jardin & %.2f m3 pour la maison)" % ((df['eau consommée pour le jardin (citerne)'].sum() + df['eau consommée pour la maison'].sum()), df['eau consommée pour le jardin (citerne)'].sum(), df['eau consommée pour la maison'].sum()) \
    + "</br>%.2f m3 d'eau perdu de trop plein" % df['perte de trop plein (citerne)'].sum() \
    + '</br>===== Cuves' \
    + "</br>%.2f m3 d'eau collectée" % df['eau collectées ce jour (cuves)'].sum() \
    + "</br>%.2f m3 d'eau utilisés pour le jardin (inclus %.d%% de perte)" % (df['eau collectées ce jour (cuves)'].sum() - df['perte de trop plein (cuves)'].sum(), w_percent_perte_cuves.value) \
    + "</br>%.2f m3 d'eau perdu de trop plein" % df['perte de trop plein (cuves)'].sum() \
    + "</br>===== Consommation d'eau pour le jardin" \
    + "</br>%.2f m3 (%.2f m3 des cuves & %.2f m3 de la citerne)" % (df['eau consommée pour le jardin'].sum(), df['eau consommée pour le jardin (cuves)'].sum(), df['eau consommée pour le jardin (citerne)'].sum()) \
    + "</br>%.2f m3 du secteur (dont %.2f m3 en juillet-aout)" % (df['eau consommée du secteur pour le jardin'].sum(), df[df['mois'].isin(['July', 'August'])]['eau consommée du secteur pour le jardin'].sum()) \
    + "</br>===== Consommation d'eau pour la maison" \
    + "</br>%.2f m3 de la citerne sur les %.2f m3 estimée (%.1f%%)" % (df['eau consommée pour la maison'].sum(), m3_moyen_journalier_maison*len(df), (df['eau consommée pour la maison'].sum()/(m3_moyen_journalier_maison*365))*100) \
    + "</br>%.2f m3 du secteur" % df['eau consommée du secteur pour la maison'].sum()
    # TODO part dans le mois subvenue au besoin

In [5]:
# années de la simulation # TODO ToggleButtons ?
w_annee = widgets.SelectMultiple(
    options=['2017', '2018', '2019', '2020'],
    value=['2020'],
    rows=4,
    disabled=False
)

def load_csv(years):
    global df
    if len(years) == 1:
        df = pd.read_csv(str(years[0]) + '.csv', sep=';')
    else:
        # Moyenne sur les années selectionnées
        df = pd.concat([pd.read_csv(str(year) + '.csv', sep=';') for year in years], axis=1)
        df = pd.DataFrame({'mois': df['mois'].mean(axis=1), 'pluviometrie': df['pluviometrie'].mean(axis=1)})
        df['mois'] = df['mois'].astype(int)
    df['mois'] = df['mois'].apply(lambda mois: calendar_month[mois])
    df['jour'] = df.groupby('mois').cumcount() + 1
    df['mois_jour'] = df.apply(lambda row: str(row['mois']) + '_' + str(row['jour']), axis=1)

def update_hist_annee():
    global df, fig_hist_annee_mois, fig_hist_annee_mois_jour
    if fig_hist_annee_mois is None:
        fig_hist_annee_mois = go.FigureWidget(px.histogram(df, x="mois", y="pluviometrie"))
    if fig_hist_annee_mois_jour is None:
        fig_hist_annee_mois_jour = go.FigureWidget(px.histogram(df, x="mois_jour", y="pluviometrie"))
    with fig_hist_annee_mois.batch_update():
        fig_hist_annee_mois.data[0].x = df['mois']
        fig_hist_annee_mois.data[0].y = df['pluviometrie']
    with fig_hist_annee_mois_jour.batch_update():
        fig_hist_annee_mois_jour.data[0].x = df['mois_jour']
        fig_hist_annee_mois_jour.data[0].y = df['pluviometrie']

def refresh_all(change):
    load_csv(change['new'])
    refresh_m2_jardin({'new': w_m2_jardin.value})
    update_m3_moyen_journalier_maison({'new': w_m3_annuelle_maison.value})
    update_m3_stockage_cuves({'new': w_m3_stockage_cuves.value})
    update_m3_stockage_citerne({'new': w_m3_stockage_citerne.value})
    update_hist_annee()
    refresh_data_plot_simulation('')
    update_hist_conso_jardin('')

w_annee.observe(refresh_all, names='value')

w_config_rapide = widgets.ToggleButtons(
    options=['Slow', 'Regular', 'Fast'],
    description='Speed:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast']
)

# cuves
w_m2_garage = widgets.FloatSlider(
    value=45,
    min=0,
    max=100.0,
    step=0.1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
stockage_cuves = {
    'buanderie': 1,
    'garage': 2,
    'jardin noir': 1,
    'jardin marron': 2*0.3,
    #'cabanon': 2
}
w_m3_stockage_cuves = widgets.SelectMultiple(
    options=[str(cuve) + ' (' + str(m3) + ' m3)' for cuve, m3 in stockage_cuves.items()],
    value=[str(cuve) + ' (' + str(m3) + ' m3)' for cuve, m3 in stockage_cuves.items()],
    rows=len(stockage_cuves),
    disabled=False
)
def update_m3_stockage_cuves(change):
    global m3_stockage_cuves
    m3_stockage_cuves = sum([stockage_cuves[cuve.split(' (')[0]] for cuve in change['new']])
#     w_stock_faible_cuves.max = m3_stockage_cuves
    # TODO add in label like besoin_eau_jardin_jour
w_m3_stockage_cuves.observe(update_m3_stockage_cuves, names='value')

w_mois_vides_cuves = widgets.SelectionRangeSlider(
    options=list(np.roll(calendar_month.values(), 6)),
    value=('November', 'February'),
    disabled=False
)
def get_months_between_in_widget(widget):
    options = widget.options
    start, end = widget.value
    return [start] + list(options[len(options) - options[::-1].index(start) : options.index(end)]) + [end]

w_percent_perte_cuves = widgets.IntSlider(
    value=5,
    min=0,
    max=100,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

# citerne
w_m2_maison = widgets.FloatSlider(
    value=67,
    min=0,
    max=200.0,
    step=0.1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
stockage_citerne = {
    'citerne': 10,
    'bassin': 5
}
w_m3_stockage_citerne = widgets.SelectMultiple(
    options=[str(cuve) + ' (' + str(m3) + ' m3)' for cuve, m3 in stockage_citerne.items()],
    value=[str(cuve) + ' (' + str(m3) + ' m3)' for cuve, m3 in stockage_citerne.items()],
    rows=len(stockage_citerne),
    disabled=False
)
def update_m3_stockage_citerne(change):
    global m3_stockage_citerne
    m3_stockage_citerne = sum([stockage_citerne[citerne.split(' (')[0]] for citerne in change['new']])
    w_reste_jour_precedent_citerne.max = m3_stockage_citerne
    w_stock_faible_citerne.max = m3_stockage_citerne
    # TODO add in label like besoin_eau_jardin_jour

w_m3_stockage_citerne.observe(update_m3_stockage_citerne, names='value')
w_reste_jour_precedent_citerne = widgets.IntSlider(
    value=0,
    min=0,
    max=sum(stockage_citerne.values()),
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
w_citerne_pour_conso_maison = widgets.Checkbox(
    value=False,
    disabled=False,
    indent=True
)
w_citerne_pour_conso_jardin = widgets.Checkbox(
    value=True,
    disabled=False,
    indent=True
)

w_mois_citerne_priorite_jardin = widgets.SelectionRangeSlider(
    options=calendar_month.values(),
    value=('April', 'September'),
    disabled=not (w_citerne_pour_conso_jardin.value == True and w_citerne_pour_conso_maison.value == True)
)
w_stock_faible_citerne = widgets.FloatSlider(
    value=0,
    min=0,
    max=sum(stockage_citerne.values()),
    step=0.1,
    disabled=not (w_citerne_pour_conso_jardin.value == True and w_citerne_pour_conso_maison.value == True),
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
def disable_jardin_options(change):
    w_mois_citerne_priorite_jardin.disabled = not (w_citerne_pour_conso_jardin.value == True and w_citerne_pour_conso_maison.value == True)
    w_stock_faible_citerne.disabled = not (w_citerne_pour_conso_jardin.value == True and w_citerne_pour_conso_maison.value == True)
        
w_citerne_pour_conso_jardin.observe(disable_jardin_options)
w_citerne_pour_conso_maison.observe(disable_jardin_options)

# consommation maison
w_m3_annuelle_maison = widgets.IntSlider(
    value=100,
    min=0,
    max=200,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
def update_m3_moyen_journalier_maison(change):
    global m3_moyen_journalier_maison
    m3_moyen_journalier_maison = change['new'] / 365.0
    # TODO add in label like besoin_eau_jardin_jour

w_m3_annuelle_maison.observe(update_m3_moyen_journalier_maison, names='value')

# besoin d'eau
w_mois_le_plus_consommateur = widgets.SelectionRangeSlider(
    options=calendar_month.values(),
    value=('June', 'July')
)

w_std_besoin_eau = widgets.FloatSlider(
    value=1.8,
    min=0.1,
    max=10,
    step=0.1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

def besoin_eau(mois, mois_le_plus_consommateur, std_besoin_eau):
    mois = reversed_calendar_month[mois]
    mois_le_plus_consommateurs = [reversed_calendar_month[m] for m in mois_le_plus_consommateur]
    return max([np.exp(-(float(((mois-mois_le_plus_consommateur-6)%12)-6) / float(std_besoin_eau))**2.0) for mois_le_plus_consommateur in mois_le_plus_consommateurs]) # TODO optim

def df_plot_besoin_eau(besoin_eau, w_mois_le_plus_consommateur, std_besoin_eau):
    df_besoin_eau = []
    for month in range(1, 13):
        df_besoin_eau.append([month, besoin_eau(calendar_month[month], get_months_between_in_widget(w_mois_le_plus_consommateur), std_besoin_eau) * 100])
    return pd.DataFrame(df_besoin_eau, columns=['Mois', '%'])

def plot_besoin_eau(df_besoin_eau):
    fig = go.FigureWidget(px.line(df_besoin_eau, x="Mois", y="%", title="% estimée du besoin d'eau par mois".decode("utf-8")))
    fig.update_yaxes(range=[0, 100], row=1, col=1)
    fig.update_xaxes(ticktext=calendar_month.values(), tickvals=list(range(1, 13)))
    return fig

def update_besoin_eau(change):
    global fig_plot_besoin_eau
    df_besoin_eau = df_plot_besoin_eau(besoin_eau, w_mois_le_plus_consommateur, w_std_besoin_eau.value)
    with fig_plot_besoin_eau.batch_update():
        fig_plot_besoin_eau.data[0].y = df_besoin_eau['%']

w_mois_le_plus_consommateur.observe(update_besoin_eau, names='value')
w_std_besoin_eau.observe(update_besoin_eau, names='value')
df_besoin_eau = df_plot_besoin_eau(besoin_eau, w_mois_le_plus_consommateur, w_std_besoin_eau.value)
fig_plot_besoin_eau = plot_besoin_eau(df_besoin_eau)

# consommation eau jardin
w_m2_jardin = widgets.FloatSlider(
    value=140,
    min=0,
    max=300.0,
    step=0.1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
w_nombre_maximum_periode_sans_eau = widgets.IntSlider(
    value=10,
    min=1,
    max=15,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
w_min_pluviometrie_vitale = widgets.FloatSlider(
    value=2.5,
    min=0,
    max=30.0,
    step=0.01,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
# w_stock_faible_cuves = widgets.FloatSlider(
#     value=0.5,
#     min=0,
#     max=sum(stockage_cuves.values()),
#     step=0.01,
#     disabled=False,
#     continuous_update=False,
#     orientation='horizontal',
#     readout=True,
#     readout_format='.2f',
# )

def arrosage(nombre_jour_sans_eau, nombre_maximum_periode_sans_eau):
    return 1/(1+(nombre_jour_sans_eau/(1-nombre_jour_sans_eau+(nombre_maximum_periode_sans_eau-1)))**-3)

def df_plot_arrosage(arrosage, nombre_maximum_periode_sans_eau, eps=1e-10):
    df_arrosage = []
    df_arrosage.append([0, 0])
    for day in range(1, nombre_maximum_periode_sans_eau + 1):
        df_arrosage.append([day, arrosage(day + eps, nombre_maximum_periode_sans_eau) * 100])
    return pd.DataFrame(df_arrosage, columns=['Nombre de jours sans pluie', '%'])

def plot_arrosage(df_arrosage):
    fig = go.FigureWidget(px.line(df_arrosage, x="Nombre de jours sans pluie", y="%", title='% de cuve videe pour arroser par jour, en fonction du nombre de jours sans pluie'))
    return fig

def update_hist_conso_jardin(change):
    global fig_percent_conso_jardin
    if fig_percent_conso_jardin is None:
        fig_percent_conso_jardin = go.FigureWidget(px.histogram(df, x="mois_jour", y="conso jardin (%)"))
    with fig_percent_conso_jardin.batch_update():
        fig_percent_conso_jardin.data[0].x = df['mois_jour']
        fig_percent_conso_jardin.data[0].y = df['conso jardin (%)']

def update_plot_arrosage(change):
    global fig_plot_arrosage
    df_arrosage = df_plot_arrosage(arrosage, change['new'], eps=1e-10)
    with fig_plot_arrosage.batch_update():
        fig_plot_arrosage.data[0].x = df_arrosage['Nombre de jours sans pluie']
        fig_plot_arrosage.data[0].y = df_arrosage['%']
        fig_plot_arrosage.update_xaxes(range=[0, change['new']], row=1, col=1)
        fig_plot_arrosage.update_yaxes(range=[0, 100], row=1, col=1)
    update_hist_conso_jardin('')

def refresh_m2_jardin(change):
    global besoin_eau_jardin_jour
    besoin_eau_jardin_jour = (w_min_pluviometrie_vitale.value * float(change['new'])) / 1000.0

w_nombre_maximum_periode_sans_eau.observe(update_plot_arrosage, names='value')
w_min_pluviometrie_vitale.observe(update_hist_conso_jardin, names='value')
w_m2_jardin.observe(refresh_m2_jardin, names='value')
df_arrosage = df_plot_arrosage(arrosage, w_nombre_maximum_periode_sans_eau.value)
fig_plot_arrosage = plot_arrosage(df_arrosage)

# output
output_metrics = widgets.HTML(
    value="",
    placeholder='',
    description='',
)

def refresh_data_plot_simulation(change):
    global fig_simulation
    simulation()
    if fig_simulation is None:
        fig_simulation = plot_simulation()
    else:
        plots = get_plots_simulation()
        with fig_simulation.batch_update():
            for d in range(len(fig_simulation.data)):
                fig_simulation.data[d].x = plots[d][0]['x']
                fig_simulation.data[d].y = plots[d][0]['y']
                
    output_metrics.value = print_metrics().decode("utf-8")

w_nombre_maximum_periode_sans_eau.observe(refresh_data_plot_simulation, names='value')
w_m2_jardin.observe(refresh_data_plot_simulation, names='value')
w_min_pluviometrie_vitale.observe(refresh_data_plot_simulation, names='value')
# w_stock_faible_cuves.observe(refresh_data_plot_simulation, names='value')
w_m2_garage.observe(refresh_data_plot_simulation, names='value')
w_mois_vides_cuves.observe(refresh_data_plot_simulation, names='value')
w_percent_perte_cuves.observe(refresh_data_plot_simulation, names='value')
w_m2_maison.observe(refresh_data_plot_simulation, names='value')
w_reste_jour_precedent_citerne.observe(refresh_data_plot_simulation, names='value')
w_mois_citerne_priorite_jardin.observe(refresh_data_plot_simulation, names='value')
w_citerne_pour_conso_jardin.observe(refresh_data_plot_simulation, names='value')
w_citerne_pour_conso_jardin.observe(refresh_data_plot_simulation, names='value')
w_citerne_pour_conso_maison.observe(refresh_data_plot_simulation, names='value')
w_stock_faible_citerne.observe(refresh_data_plot_simulation, names='value')
w_m3_stockage_cuves.observe(refresh_data_plot_simulation, names='value')
w_m3_stockage_citerne.observe(refresh_data_plot_simulation, names='value')
w_mois_le_plus_consommateur.observe(refresh_data_plot_simulation, names='value')
w_std_besoin_eau.observe(refresh_data_plot_simulation, names='value')

# labels
widget_labels = {
    w_annee: 'Année de la simulation'.decode("utf-8"),
    w_nombre_maximum_periode_sans_eau: 'Nombre maximum de jours sans pluie :'.decode("utf-8"),
    #w_max_eau_consomme: "m3 d'eau par jour necessaure pour arroser le jardin (TODO = w_min_pluviometrie_vitale * m2 jardin / 1000 ?)", # TODO si oui, ajouter m2 de jardin et label estimation w_max_eau_consomme
    w_min_pluviometrie_vitale: "Pluviometrie (mm/m2) par jour minimum requise pour le jardin".decode("utf-8"),
#     w_stock_faible_cuves: "Seuil (m3) a partir duquel on utilise l'eau du secteur".decode("utf-8"),
    w_m2_garage: 'Mètres carrés de garage'.decode("utf-8"),
    w_mois_vides_cuves: 'Cuves vidées'.decode("utf-8"),
    w_percent_perte_cuves: "% de pertes d'eau (fuites, joints, transfert, etc.)".decode("utf-8"),
    w_m2_maison: 'Mètres carrés de la maison'.decode("utf-8"),
    w_reste_jour_precedent_citerne: "w_reste_jour_precedent_citerne".decode("utf-8"),
    w_mois_citerne_priorite_jardin: 'w_mois_citerne_priorite_jardin'.decode("utf-8"),
    w_citerne_pour_conso_jardin: 'Citerne utilisée pour le jardin'.decode("utf-8"),
    w_citerne_pour_conso_maison: 'Citerne utilisée pour la maison'.decode("utf-8"),
    w_stock_faible_citerne: 'Mètres cubes restants de la citerne reservés au jardin'.decode("utf-8"),
    w_m2_jardin: 'Mètres carrés de jardin'.decode("utf-8"),
    w_m3_stockage_cuves: 'Cuves'.decode("utf-8"),
    w_mois_le_plus_consommateur: "Mois les plus consommateur d'eau de l'année".decode("utf-8"),
    w_std_besoin_eau: "Etalement du besoin d'eau".decode("utf-8"),
}

# ===========

df = None
fig_hist_annee_mois = None
fig_hist_annee_mois_jour = None
fig_percent_conso_jardin = None
fig_simulation = None
besoin_eau_jardin_jour = None
m3_moyen_journalier_maison = None
m3_stockage_cuves = None
m3_stockage_citerne = None
refresh_all({'new': w_annee.value})

# layout_item = Layout( width='auto')
layout_label = Layout(width='40%', flex_flow='wrap')
layout_hbox_dual = Layout(justify_content='space-around')
layout_menu = Layout(display='flex',
                    flex_flow='column',
                     height='500px',
                     overflow='scroll hidden',
                    align_items='stretch',
#                     border='solid',
                    width='50%')
fig_plot_besoin_eau.layout.width = 425
fig_plot_besoin_eau.layout.height = 325
fig_plot_arrosage.layout.width = 425
fig_plot_arrosage.layout.height = 325

tab_1 = VBox([HBox([Label(widget_labels[widget] if widget in widget_labels.keys() else '', layout=layout_label), widget], layout=layout_hbox_dual) 
              for widget in [w_annee]]
            + [w_config_rapide])
tab_2 = VBox([HBox([Label(widget_labels[widget] if widget in widget_labels.keys() else '', layout=layout_label), widget], layout=layout_hbox_dual) 
              for widget in [w_m2_garage, w_m2_maison]]
            + [fig_hist_annee_mois, fig_hist_annee_mois_jour])
tab_3 = VBox([HBox([Label(widget_labels[widget] if widget in widget_labels.keys() else '', layout=layout_label), widget], layout=layout_hbox_dual) 
              for widget in [w_m3_stockage_cuves, w_mois_vides_cuves, w_percent_perte_cuves, w_m3_stockage_citerne, w_reste_jour_precedent_citerne, w_citerne_pour_conso_jardin, w_citerne_pour_conso_maison, w_mois_citerne_priorite_jardin, w_stock_faible_citerne]]) # w_stock_faible_cuves
tab_4 = VBox([HBox([Label(widget_labels[w_m2_jardin], layout=layout_label), w_m2_jardin], layout=layout_hbox_dual)]
            + [HBox([fig_plot_besoin_eau,
                     VBox([Label("Mois les plus consommateur d'eau de l'année :".decode("utf-8")), 
                           w_mois_le_plus_consommateur,
                           Label("Etalement du besoin d'eau :".decode("utf-8")), 
                           w_std_besoin_eau], 
                           layout=layout_hbox_dual)])]
             + [HBox([fig_plot_arrosage,
                      VBox([Label("Nombre maximum de jours sans pluie :".decode("utf-8")), 
                           w_nombre_maximum_periode_sans_eau,
                           Label("Pluviometrie (mm/m2) par jour minimum requise pour le jardin".decode("utf-8")), 
                           w_min_pluviometrie_vitale,
                          ], 
                          layout=layout_hbox_dual)])]
            + [fig_percent_conso_jardin])

input_simulation = widgets.Tab([tab_1, tab_2, tab_3, tab_4], layout=layout_menu)
input_simulation.set_title(0, 'Années'.decode("utf-8"))
input_simulation.set_title(1, 'Collecte')
input_simulation.set_title(2, 'Stockage')
input_simulation.set_title(3, 'Consommation jardin')

haut = HBox([input_simulation, output_metrics])
VBox([haut, fig_simulation])

# todo add check box w_reste_jour_precedent_citerne "utiliser valeur decembre"
# dans "année ajouter raccourci config interessantes"
# mise en forme des metriques
# pluviométrie: eau cumullé a droite de la sum de pluvio
# jeu de couleur et legende bien definit et checkbox pour avoir echelle loga
# idéalement, prendre en compte besoin_eau_croissance avec la secheresse du sol (vent + ensolleillement + température + rosé + hygrométrie (humidité dans l'air))
# rosé = gradient (diff min et diff max)
# besoin par plantes
# arrosage dépend du plantage
# cout electriques pompes

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oVGFiKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShMYWJlbCh2YWx1ZT11J0Fublx4ZTllIGRlIGxhIHNpbXVsYXTigKY=


In [6]:
# df['cumul eau collectées (cuves)'] = df['eau collectées ce jour (cuves)'].cumsum()
# df['cumul eau collectées (citerne)'] = df['eau collectées ce jour (citerne)'].cumsum()
# df.loc[df['mois'].isin(w_mois_vides_cuves.value), ['cumul eau collectées (cuves)']] = 0
# 
# # figure_data=[]
# # figure_data.extend([i for i in px.histogram(df, x="mois_jour", y="cumul eau collectées (cuves)", color=['cumul eau collectées (cuves)'] * len(df)).to_dict()['data']])
# # figure_data.extend([i for i in px.histogram(df, x="mois_jour", y="cumul eau collectées (citerne)", color=['cumul eau collectées (citerne)'] * len(df)).to_dict()['data']])
# # fig=go.Figure(figure_data)
# # fig.update_layout(barmode='stack')
# # fig.show()

In [7]:
# deploy ec2 machine